<a href="https://colab.research.google.com/github/mtyin/spanner-graph-agent/blob/main/docs/Chat_with_Spanner_Graph_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overview

This colab walks through how to chat with your Spanner Graph using natural language and provide some tips on addressing some common practical issues along the way.



# Setup

In [ ]:
# @title Authentication

from google.colab import auth
auth.authenticate_user()

In [ ]:
# @title Install the Spanner Graph Agent package
!pip install --force-reinstall --no-deps langchain-google-spanner@git+https://github.com/mtyin/langchain-google-spanner-python@multiplex # Delete this line when langchain-google-spanner is released with new version
!pip install --quiet git+https://github.com/mtyin/spanner-graph-agent.git

In [ ]:
# @title Set up environment
# @markdown Please fill in the value below with your Google Cloud project ID and then run the cell.

PROJECT_ID = ""    # @param {type:"string"}
INSTANCE_ID = ""   # @param {type:"string"}
DATABASE_ID = ""   # @param {type:"string"}
DEFAULT_LLM_MODEL_NAME = "gemini-2.5-flash"                 # @param {type:"string"}
DEFAULT_EMBEDDING_MODEL_NAME = "text-embedding-004"         # @param {type:"string"}
LOCATION = "us-central1"                                    # @param {type:"string"}
LOG_LEVEL = 'DEBUG'                                          #@param ["", "INFO", "DEBUG"]

# Set the project id
!gcloud config set project {PROJECT_ID}
%env GOOGLE_CLOUD_PROJECT={PROJECT_ID}

# Set the envionrment variables
import os
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'TRUE'
os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID
os.environ['GOOGLE_CLOUD_LOCATION'] = LOCATION


# Suppress unnecessary warning from the dependency library.
import logging
class _NoFunctionCallWarning(logging.Filter):
    def filter(self, record: logging.LogRecord) -> bool:
        message = record.getMessage()
        if "there are non-text parts in the response:" in message:
            return False
        else:
            return True

logging.getLogger("google_genai.types").addFilter(_NoFunctionCallWarning())

In [ ]:
# @title: Download the sample dataset
!curl -L -o finance_data.tar.gz https://raw.githubusercontent.com/mtyin/spanner-graph-agent/main/datasets/finance/finance_data.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1590k  100 1590k    0     0  2827k      0 --:--:-- --:--:-- --:--:-- 2825k


In [ ]:
# @title: Load the sample dataset
from spanner_graph_agent.utils.dataset import Dataset

dataset = Dataset("finance_data.tar.gz")
dataset.load(INSTANCE_ID, DATABASE_ID, PROJECT_ID)

In [ ]:
# @title Show sample graph
%%spanner_graph --project {PROJECT_ID} --instance {INSTANCE_ID} --database {DATABASE_ID}

GRAPH FinanceGraph
MATCH p1 = (:Person) -[:worksAt]-> (c:Company),
      p2 = (c:Company) <-[:ownsShare]- (:Person),
      p3 = (c:Company) <-[:ownsShare]- (:MutualFund),
      p4 = (c:Company) <-[:ownsShare]- (:Company)
LIMIT 100
FOR p IN [p1, p2, p3, p4]
RETURN SAFE_TO_JSON(p) AS p

In [ ]:
# @title Build the Spanner Graph Agent
from spanner_graph_agent import SpannerGraphAgent
from spanner_graph_agent.utils.agent_session import AgentSession
import asyncio
import uuid
from google.genai import types
from google.adk.planners import BuiltInPlanner
from google.genai import types

GRAPH_ID='FinanceGraph'
user_id = str(uuid.uuid4())

root_agent = SpannerGraphAgent(
  instance_id=INSTANCE_ID,
  database_id=DATABASE_ID,
  graph_id=GRAPH_ID,
  model=DEFAULT_LLM_MODEL_NAME,
  agent_config={
    "verify_gql": False,
    "log_level": LOG_LEVEL,
  },
  # gemini-2.5-flash is a thinking model, you can toggle the budget
  # to balance between reaction speed and quality.
  planner=BuiltInPlanner(thinking_config=types.ThinkingConfig(thinking_budget=0)),
)

for tool in root_agent.tools:
  print('============')
  print(tool.name)
  print(tool.description)


DEBUG:spanner_graph_agent.spanner_graph_agent.tools.index_tools:Built UserProvidedCompanyReference data model:
{'properties': {'name': {'description': 'name of UserProvidedCompanyReference', 'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'UserProvidedCompanyReference', 'type': 'object'}

DEBUG:spanner_graph_agent.spanner_graph_agent.tools.index_tools:Built CompanyCanonicalReference data model:
{'properties': {'id': {'description': 'id of CanonicalCompanyReference', 'title': 'Id', 'type': 'string'}}, 'required': ['id'], 'title': 'CanonicalCompanyReference', 'type': 'object'}

DEBUG:spanner_graph_agent.spanner_graph_agent.tools.index_tools:Built search query:


    SELECT id AS id
    FROM Company
    WHERE search(name_token, @name)
    ORDER BY score(name_token, @name) + IF(name=@name, 1., 0.) DESC
    LIMIT 2
  


DEBUG:spanner_graph_agent.spanner_graph_agent.tools.index_tools:Built example query:


    SELECT id AS id, name AS name
    FROM Company
    TABLESAMPLE

resolve_Company_id_by_name
Resolves the canonical {'id'} of Company using given {'name'}.

Input: a list of reference in original query represented by UserProvidedCompanyReference(name: string);
Output: a mapping to the corresponding canonical references represented by CanonicalCompanyReference(id: string).


For example,
  Given UserProvidedCompanyReference(name='Orr-Bean'), this tool may return a mapping to [CanonicalCompanyReference(id='c2593')].
  [CanonicalCompanyReference(id='c2593')] should be used in subsequent queries to the knowledge graph.


This tool convert user-provided references (names, descriptions, titles, etc.)
into their corresponding canonical identifiers. The output of these tools
(the canonical ID) must be used in subsequent queries to the knowledge graph.

Sometimes there are more than one canonical references returned for a user-provided reference
if disambiguation is needed.
resolve_Person_id_by_name
Resolves the canonical {'id'} of Person using given {'name'}

### Chat with Spanner Graph

#### Get started

In [ ]:
# @title One shot questions

QUERY = "How many nodes in the graph?"    # @param {type:"string"}

with AgentSession(agent=root_agent, user_id=user_id) as session:
  response =  await session.ainvoke(QUERY)
  print(response.content.parts[0].text)

DEBUG:spanner_graph_agent.spanner_graph_agent.tools.langchain_tools:Input query: `How many nodes in the graph?`




> Entering new SpannerGraphQAChain chain...
Executing gql:
GRAPH FinanceGraph
MATCH (n)
RETURN COUNT(n) AS total_nodes;
Full Context:
[{'total_nodes': 10000}]

> Finished chain.
The graph has 10000 nodes.


In [ ]:
# @title Conversation with memory
import datetime

QUERY = "Who is the Chief Executive Officer in Phillips LLC?"    # @param {type:"string"}
FOLLOW_UP_QUERY = "Does she invest into other companies?" # @param {type:"string"}

with AgentSession(agent=root_agent, user_id=user_id) as session:
  response =  await session.ainvoke(QUERY)
  print(response.content.parts[0].text)

  response =  await session.ainvoke(FOLLOW_UP_QUERY)
  print(response.content.parts[0].text)

DEBUG:spanner_graph_agent.spanner_graph_agent.tools.langchain_tools:Input query: `Who is the Chief Executive Officer in Company(name='Phillips LLC')?`




> Entering new SpannerGraphQAChain chain...
Executing gql:
GRAPH FinanceGraph
MATCH (p:Person)-[w:worksAt]->(c:Company WHERE c.name = 'Phillips LLC')
WHERE w.job_title = 'Chief Executive Officer'
RETURN p.name AS ceo_name;
Full Context:
[{'ceo_name': 'Melissa Humphrey'}]

> Finished chain.
Melissa Humphrey is the Chief Executive Officer in Phillips LLC.


DEBUG:spanner_graph_agent.spanner_graph_agent.tools.langchain_tools:Input query: `Does Person(name='Melissa Humphrey') invest into other companies?`




> Entering new SpannerGraphQAChain chain...
Executing gql:
GRAPH FinanceGraph
MATCH (p:Person WHERE p.name = 'Melissa Humphrey')-[:ownsShare]->(c:Company)
RETURN p.name AS investor_name, c.name AS invested_company_name
Full Context:
[{'investor_name': 'Melissa Humphrey', 'invested_company_name': 'Mccarthy, Myers and Hernandez'}, {'investor_name': 'Melissa Humphrey', 'invested_company_name': 'Gonzales-Dixon'}, {'investor_name': 'Melissa Humphrey', 'invested_company_name': 'Walker, Jones and Howard'}]

> Finished chain.
Melissa Humphrey invests in Mccarthy, Myers and Hernandez, Gonzales-Dixon, and Walker, Jones and Howard.


#### Challenges: LLM can generate incorrect GQL query syntax

The challenge: to be elaborate.

The solution: to be elaborate.
* LLM is pretty good at generalizing over a few examples

In [ ]:
# @title An example of incorrect GQL syntax

QUERY = "Which person invests into more than 10 companies?"    # @param {type:"string"}

with AgentSession(agent=root_agent, user_id=user_id) as session:
  response =  await session.ainvoke(QUERY)
  print(response.content.parts[0].text)

DEBUG:spanner_graph_agent.spanner_graph_agent.tools.langchain_tools:Input query: `Which person invests into more than 10 companies?`




> Entering new SpannerGraphQAChain chain...
Executing gql:
GRAPH FinanceGraph
MATCH (p:Person)-[:ownsShare]->(c:Company)
GROUP BY p
HAVING COUNT(DISTINCT c) > 10
RETURN p.name as person_name
Invalid generated gql:
GRAPH FinanceGraph
MATCH (p:Person)-[:ownsShare]->(c:Company)
GROUP BY p
HAVING COUNT(DISTINCT c) > 10
RETURN p.name as person_name
Query error: 
400 Syntax error: Unexpected keyword GROUP [at 3:1]\nGROUP BY p\n^ [locale: "en-US"
message: "Syntax error: Unexpected keyword GROUP [at 3:1]\nGROUP BY p\n^"
]
Executing gql:
GRAPH FinanceGraph
MATCH (p:Person)-[:ownsShare]->(c:Company)
RETURN p, COUNT(DISTINCT c) AS num_companies_invested_in
GROUP BY p
HAVING num_companies_invested_in > 10

NEXT
RETURN p.name AS person_name;
Invalid generated gql:
GRAPH FinanceGraph
MATCH (p:Person)-[:ownsShare]->(c:Company)
RETURN p, COUNT(DISTINCT c) AS num_companies_invested_in
GROUP BY p
HAVING num_companies_invested_in > 10

NEXT
RETURN p.name AS person_name;
Query error: 
400 Syntax error: 

ERROR:spanner_graph_agent.spanner_graph_agent.tools.langchain_tools:Failed QA chain invocation: The generated gql query is invalid


I don't know which person invests in more than 10 companies.


In [ ]:
# @title Build the Spanner Graph Agent with example table

GQL_EXAMPLE_TABLE = "gql_examples"   # @param {type:"string"}
root_agent = SpannerGraphAgent(
  instance_id=INSTANCE_ID,
  database_id=DATABASE_ID,
  graph_id=GRAPH_ID,
  model=DEFAULT_LLM_MODEL_NAME,
  agent_config={
    "example_table": GQL_EXAMPLE_TABLE or None,
    "embedding": DEFAULT_EMBEDDING_MODEL_NAME,
    "verify_gql": False,
    "log_level": LOG_LEVEL,
  },
  planner=BuiltInPlanner(thinking_config=types.ThinkingConfig(thinking_budget=0)),
)

DEBUG:spanner_graph_agent.spanner_graph_agent.tools.index_tools:Built UserProvidedCompanyReference data model:
{'properties': {'name': {'description': 'name of UserProvidedCompanyReference', 'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'UserProvidedCompanyReference', 'type': 'object'}

DEBUG:spanner_graph_agent.spanner_graph_agent.tools.index_tools:Built CompanyCanonicalReference data model:
{'properties': {'id': {'description': 'id of CanonicalCompanyReference', 'title': 'Id', 'type': 'string'}}, 'required': ['id'], 'title': 'CanonicalCompanyReference', 'type': 'object'}

DEBUG:spanner_graph_agent.spanner_graph_agent.tools.index_tools:Built search query:


    SELECT id AS id
    FROM Company
    WHERE search(name_token, @name)
    ORDER BY score(name_token, @name) + IF(name=@name, 1., 0.) DESC
    LIMIT 2
  


DEBUG:spanner_graph_agent.spanner_graph_agent.tools.index_tools:Built example query:


    SELECT id AS id, name AS name
    FROM Company
    TABLESAMPLE

Waiting for operation to complete...


/usr/local/lib/python3.11/dist-packages/vertexai/_model_garden/_model_garden_models.py:278: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()
INFO:spanner_graph_agent.spanner_graph_agent.agents:Tool: resolve_Company_id_by_name
Description: Resolves the canonical {'id'} of Company using given {'name'}.

Input: a list of reference in original query represented by UserProvidedCompanyReference(name: string);
Output: a mapping to the corresponding canonical references represented by CanonicalCompanyReference(id: string).


For example,
  Given UserProvidedCompanyReference(name='Chang-Martin'), this tool may return a mapping to [CanonicalCompanyReference(id='c437')].
  [CanonicalCompanyReference(id='c437')] should be used in subsequent queries to the knowledge graph.


This tool convert user-provided 

In [ ]:
# @title Add a query example that demonstrates the basic GQL query
EXAMPLE_USER_QUERY = "Find persons that are employed by both companyNameA and companyNameB?"   # @param {type:"string"}
EXAMPLE_GQL_QUERY = """
GRAPH FinanceGraph
-- `companyNameA` and `companyNameB` are both strings
MATCH (company_a:Company {name: "companyNameA"}) <-[:worksAt]- (employee:Person),
 (company_b:Company {name: "companyNameB"}) <-[:worksAt]- (employee:Person)
RETURN employee.name
"""
root_agent.gql_query_tool.add_example(user_query=EXAMPLE_USER_QUERY, gql=EXAMPLE_GQL_QUERY)

In [ ]:
# @title Add a query example that demonstrates the GQL FILTER statement
EXAMPLE_USER_QUERY = "Which companies have more than 42 employees?"   # @param {type:"string"}
EXAMPLE_GQL_QUERY = """
GRAPH FinanceGraph
MATCH (company:Company) <-[:worksAt]- (:Person)
WITH company.name, COUNT(*) AS cnt
FILTER cnt > 42
RETURN company.name, cnt
"""
root_agent.gql_query_tool.add_example(user_query=EXAMPLE_USER_QUERY, gql=EXAMPLE_GQL_QUERY)

In [ ]:
# @title Add a query example that demonstrates the GQL subquery
EXAMPLE_USER_QUERY = "Return persons who are not affiliated with companyNameXYZ"   # @param {type:"string"}
EXAMPLE_GQL_QUERY = """
GRAPH FinanceGraph
MATCH (person:Person)
WHERE NOT EXISTS {
 MATCH (:Company {name: "companyNameXYZ"}) <-[:worksAt]- (employee:Person)
 WHERE employee = person -- use an explicit filter instead of redeclaring `person`
}
RETURN n.name
"""
root_agent.gql_query_tool.add_example(user_query=EXAMPLE_USER_QUERY, gql=EXAMPLE_GQL_QUERY)

In [ ]:
# @title Revisit the example of incorrect GQL syntax

QUERY = "Which person invests into more than 10 companies?"    # @param {type:"string"}

with AgentSession(agent=root_agent, user_id=user_id) as session:
  response =  await session.ainvoke(QUERY)
  print(response.content.parts[0].text)

DEBUG:spanner_graph_agent.spanner_graph_agent.tools.langchain_tools:Input query: `Which person invests into more than 10 companies?`




> Entering new SpannerGraphQAChain chain...
Executing gql:
GRAPH FinanceGraph
MATCH (person:Person) -[:ownsShare]-> (company:Company)
WITH person.name AS person_name, COUNT(DISTINCT company) AS num_invested_companies
FILTER num_invested_companies > 10
RETURN person_name, num_invested_companies
Full Context:
[{'person_name': 'Victoria Moore', 'num_invested_companies': 11}, {'person_name': 'Sarah Roberts', 'num_invested_companies': 11}, {'person_name': 'Tristan Horne', 'num_invested_companies': 11}, {'person_name': 'Michael Ferguson', 'num_invested_companies': 12}, {'person_name': 'Angela Thomas', 'num_invested_companies': 11}, {'person_name': 'Danielle Turner', 'num_invested_companies': 14}, {'person_name': 'Brian Martin', 'num_invested_companies': 11}, {'person_name': 'Nicholas Thompson', 'num_invested_companies': 12}, {'person_name': 'Michael Jones', 'num_invested_companies': 15}, {'person_name': 'Megan Terry', 'num_invested_companies': 16}, {'person_name': 'Lori Ross', 'num_investe

#### Challenges: LLM doesn't know the canonical reference

The challenge: to be elaborate.
- uncanonical reference
- ambiguous reference

The solution: to be elaborate.
* Spanner has many built-in tools that can help us such as full-text search.

In [ ]:
# @title Revisit the example of uncanonical and ambiguous reference

QUERY = "Show me about Aaron's employment?"    # @param {type:"string"}

with AgentSession(agent=root_agent, user_id=user_id) as session:
  response =  await session.ainvoke(QUERY)
  print(response.content.parts[0].text)

DEBUG:spanner_graph_agent.spanner_graph_agent.tools.langchain_tools:Input query: `Aaron's employment`




> Entering new SpannerGraphQAChain chain...
Executing gql:
GRAPH FinanceGraph
MATCH (person:Person {name: "Aaron"}) -[:worksAt]-> (company:Company)
RETURN company.name AS CompanyName
Full Context:
[]

> Finished chain.
I don't know the answer.


In [ ]:
# @title Show sample graph: there are multiple persons whose names contain `Aaron`
%%spanner_graph --project {PROJECT_ID} --instance {INSTANCE_ID} --database {DATABASE_ID}

GRAPH FinanceGraph
MATCH (p:Person)
WHERE REGEXP_CONTAINS(p.name, 'Aaron')
RETURN SAFE_TO_JSON(p) AS p

In [ ]:
# @title Add a full-text search index to help resolve Person reference
from google.cloud import spanner

DATABASE = spanner.Client(project=PROJECT_ID).instance(INSTANCE_ID).database(DATABASE_ID)
search_index_statements = [
    "ALTER TABLE Company ADD COLUMN IF NOT EXISTS name_token TOKENLIST AS (TOKENIZE_FULLTEXT(name)) HIDDEN",
    "CREATE SEARCH INDEX CompanyNameSearchIndex ON Company(name_token)",
]
try:
  operation = DATABASE.update_ddl(search_index_statements)
  operation.result()
except Exception as e:
  # Handle the case when search index already exists.
  print(e)

In [ ]:
# @title Add a full-text search index to help resolve Person reference

search_index_statements = [
    "ALTER TABLE Person ADD COLUMN IF NOT EXISTS name_token TOKENLIST AS (TOKENIZE_SUBSTRING(name, ngram_size_min=>3, ngram_size_max=>4)) HIDDEN",
    "CREATE SEARCH INDEX PersonFullNameSearchIndex ON Person(name_token)",
]
try:
  operation = DATABASE.update_ddl(search_index_statements)
  operation.result()
except Exception as e:
  # Handle the case when search index already exists.
  print(e)

In [ ]:
# @title Build the Spanner Graph Agent with search indexes

root_agent = SpannerGraphAgent(
  instance_id=INSTANCE_ID,
  database_id=DATABASE_ID,
  graph_id=GRAPH_ID,
  model=DEFAULT_LLM_MODEL_NAME,
  agent_config={
    "example_table": GQL_EXAMPLE_TABLE or None,
    "embedding": DEFAULT_EMBEDDING_MODEL_NAME,
    "verify_gql": False,
    "log_level": LOG_LEVEL,
  },
)

for tool in root_agent.tools:
  print('============')
  print(tool.name)
  print(tool.description)

DEBUG:spanner_graph_agent.spanner_graph_agent.tools.index_tools:Built UserProvidedCompanyReference data model:
{'properties': {'name': {'description': 'name of UserProvidedCompanyReference', 'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'UserProvidedCompanyReference', 'type': 'object'}

DEBUG:spanner_graph_agent.spanner_graph_agent.tools.index_tools:Built CompanyCanonicalReference data model:
{'properties': {'id': {'description': 'id of CanonicalCompanyReference', 'title': 'Id', 'type': 'string'}}, 'required': ['id'], 'title': 'CanonicalCompanyReference', 'type': 'object'}

DEBUG:spanner_graph_agent.spanner_graph_agent.tools.index_tools:Built search query:


    SELECT id AS id
    FROM Company
    WHERE search(name_token, @name)
    ORDER BY score(name_token, @name) + IF(name=@name, 1., 0.) DESC
    LIMIT 2
  


DEBUG:spanner_graph_agent.spanner_graph_agent.tools.index_tools:Built example query:


    SELECT id AS id, name AS name
    FROM Company
    TABLESAMPLE

Waiting for operation to complete...


/usr/local/lib/python3.11/dist-packages/vertexai/_model_garden/_model_garden_models.py:278: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()
INFO:spanner_graph_agent.spanner_graph_agent.agents:Tool: resolve_Company_id_by_name
Description: Resolves the canonical {'id'} of Company using given {'name'}.

Input: a list of reference in original query represented by UserProvidedCompanyReference(name: string);
Output: a mapping to the corresponding canonical references represented by CanonicalCompanyReference(id: string).


For example,
  Given UserProvidedCompanyReference(name='Thompson-Davis'), this tool may return a mapping to [CanonicalCompanyReference(id='c2193')].
  [CanonicalCompanyReference(id='c2193')] should be used in subsequent queries to the knowledge graph.


This tool convert user-provi

resolve_Company_id_by_name
Resolves the canonical {'id'} of Company using given {'name'}.

Input: a list of reference in original query represented by UserProvidedCompanyReference(name: string);
Output: a mapping to the corresponding canonical references represented by CanonicalCompanyReference(id: string).


For example,
  Given UserProvidedCompanyReference(name='Thompson-Davis'), this tool may return a mapping to [CanonicalCompanyReference(id='c2193')].
  [CanonicalCompanyReference(id='c2193')] should be used in subsequent queries to the knowledge graph.


This tool convert user-provided references (names, descriptions, titles, etc.)
into their corresponding canonical identifiers. The output of these tools
(the canonical ID) must be used in subsequent queries to the knowledge graph.

Sometimes there are more than one canonical references returned for a user-provided reference
if disambiguation is needed.
resolve_Person_id_by_name
Resolves the canonical {'id'} of Person using given {'

In [ ]:
# @title Revisit the example of uncanonical and ambiguous reference

QUERY = "Show me about Aaron's employment?"    # @param {type:"string"}
FOLLOWUP_QUERY = "I'm talking about Aaron Cain"    # @param {type:"string"}

with AgentSession(agent=root_agent, user_id=user_id) as session:
  response =  await session.ainvoke(QUERY)
  print(response.content.parts[0].text)

  response =  await session.ainvoke(FOLLOWUP_QUERY)
  print(response.content.parts[0].text)

DEBUG:spanner_graph_agent.spanner_graph_agent.tools.index_tools:Resolving: [UserProvidedPersonReference(name='Aaron')]...
DEBUG:spanner_graph_agent.spanner_graph_agent.tools.index_tools:Resolved reference_mappings: [ReferenceMapping(reference_in_user_query=UserProvidedPersonReference(name='Aaron'), canonical_references=[CanonicalPersonReference(id='p3718'), CanonicalPersonReference(id='p2063')])]


There are two persons named Aaron, which one are you looking for?
* Aaron (ID: p3718)
* Aaron (ID: p2063)


DEBUG:spanner_graph_agent.spanner_graph_agent.tools.index_tools:Resolving: [UserProvidedPersonReference(name='Aaron Cain')]...
DEBUG:spanner_graph_agent.spanner_graph_agent.tools.index_tools:Resolved reference_mappings: [ReferenceMapping(reference_in_user_query=UserProvidedPersonReference(name='Aaron Cain'), canonical_references=[CanonicalPersonReference(id='p891')])]
DEBUG:spanner_graph_agent.spanner_graph_agent.tools.langchain_tools:Input query: `What is the employment of Person(id='p891')?`




> Entering new SpannerGraphQAChain chain...
Executing gql:
GRAPH FinanceGraph
MATCH (p:Person {id: "p891"}) -[w:worksAt]-> (c:Company)
RETURN c.name AS companyName, w.job_title AS jobTitle
Full Context:
[{'companyName': 'Lopez PLC', 'jobTitle': 'Microbiologist'}, {'companyName': 'Brown LLC', 'jobTitle': 'Naval architect'}, {'companyName': 'Levine LLC', 'jobTitle': 'Education administrator'}, {'companyName': 'Phillips-Smith', 'jobTitle': 'Presenter, broadcasting'}, {'companyName': 'Rojas, Moon and Glover', 'jobTitle': 'Industrial/product designer'}, {'companyName': 'Cole Inc', 'jobTitle': 'Counsellor'}, {'companyName': 'Williams PLC', 'jobTitle': 'Energy manager'}, {'companyName': 'Hoffman-Martinez', 'jobTitle': 'Patent attorney'}, {'companyName': 'Humphrey LLC', 'jobTitle': 'Printmaker'}, {'companyName': 'Pineda, Aguilar and Howard', 'jobTitle': 'Field trials officer'}]

> Finished chain.
Person(id='p891') is employed as a Microbiologist at Lopez PLC, a Naval architect at Brown LLC, 

### Evaluation

The dataset also comes with a list of evaluation queries.

In [ ]:
# @title Show evaluation queries
from spanner_graph_agent.utils.dataset import Dataset

dataset = Dataset("finance_data.tar.gz")
all_templates = dataset.load_evalution_templates()

for topic, templates in all_templates.items():
  print('#', topic)
  for template in templates:
    print('## User queries:')
    for i, question in enumerate(template.get('Questions', [])):
      print(f'{i}.', question)
    print()
    print('## Reference answer query:')
    print(template.get('Answer', ''))

# 1-Hop Queries: Person to Company (Employment)
## User queries:
0. Which company does {person_name} work for?
1. What is the name of the company where {person_name} is employed?
2. Where does {person_name} work?

## Reference answer query:
GRAPH FinanceGraph
MATCH (n:Person) -[:worksAt]-> (c:Company)
WHERE n.name = @person_name
RETURN c.name

## User queries:
0. What is {person_name}''s job title?
1. What is the role of {person_name} at their job?
2. Can you tell me about {person_name}''s position?

## Reference answer query:
GRAPH FinanceGraph
MATCH (n:Person) -[e:worksAt]-> (:Company)
WHERE n.name = @person_name
RETURN e.job_title

## User queries:
0. Tell me about {person_name}''s employment.
1. Show me the employment details for {person_name}.
2. Provide information on {person_name}''s job.

## Reference answer query:
GRAPH FinanceGraph
MATCH (n:Person) -[e:worksAt]-> (c:Company)
WHERE n.name = @person_name
RETURN e.job_title, c.name AS company_name

# 1-Hop Queries: Person to Com

In [ ]:
# @title Show evaluation results

from google.cloud.spanner_v1 import param_types

num_examples_to_show = 3
cnt = 0
dataset.register_parameter_provider('person_name', lambda: ('Anita Hoffman', param_types.STRING))
dataset.register_parameter_provider('company_name', lambda: ('Grimes, Bowman and Greer', param_types.STRING))
dataset.register_parameter_provider('mutual_fund_name', lambda: ('Cultural Easy Fund', param_types.STRING))
dataset.register_parameter_provider('job_title', lambda: ('Banker', param_types.STRING))
dataset.register_parameter_provider('person_name_1', lambda: ('Anita Hoffman', param_types.STRING))
dataset.register_parameter_provider('person_name_2', lambda: ('Dr. Bobby Rogers', param_types.STRING))
dataset.register_parameter_provider('year', lambda: (2021, param_types.INT64))
async for topic, example in dataset.evaluate(root_agent, INSTANCE_ID, DATABASE_ID, PROJECT_ID):
  if cnt >= num_examples_to_show:
    break
  cnt += 1
  print('#', topic)
  print('## Question')
  print(example['question'])
  print('## Agent Answer')
  print(example['agent_answer'])
  print('## Reference Answer')
  print(example['reference_answer'])

DEBUG:spanner_graph_agent.spanner_graph_agent.tools.index_tools:Resolving: [UserProvidedPersonReference(name='Anita Hoffman')]...
DEBUG:spanner_graph_agent.spanner_graph_agent.tools.index_tools:Resolved reference_mappings: [ReferenceMapping(reference_in_user_query=UserProvidedPersonReference(name='Anita Hoffman'), canonical_references=[CanonicalPersonReference(id='p2005')])]
DEBUG:spanner_graph_agent.spanner_graph_agent.tools.langchain_tools:Input query: `Which company does Person(id='p2005') work for?`




> Entering new SpannerGraphQAChain chain...
Executing gql:
GRAPH FinanceGraph
MATCH (person:Person {id: "p2005"}) -[:worksAt]-> (company:Company)
RETURN company.name AS company_name
Full Context:
[{'company_name': 'Grimes, Bowman and Greer'}, {'company_name': 'Anderson-Briggs'}, {'company_name': 'Davila-Robinson'}, {'company_name': 'Anderson, Aguirre and Eaton'}, {'company_name': 'Cisneros Inc'}, {'company_name': 'Brown-Villanueva'}, {'company_name': 'Clark-Jones'}, {'company_name': 'Vasquez-Randolph'}, {'company_name': 'Flynn LLC'}, {'company_name': 'Pena-Diaz'}, {'company_name': 'Jenkins LLC'}, {'company_name': 'Matthews-Vega'}, {'company_name': 'Horne-Russo'}, {'company_name': 'George Ltd'}, {'company_name': 'Rogers, Wilson and Bradshaw'}, {'company_name': 'Flores Inc'}, {'company_name': 'Lyons-Zavala'}, {'company_name': 'Smith and Sons'}, {'company_name': 'Guerra-Harper'}]

> Finished chain.
# 1-Hop Queries: Person to Company (Employment)
## Question
Which company does Anita Hoff

DEBUG:spanner_graph_agent.spanner_graph_agent.tools.index_tools:Resolving: [UserProvidedPersonReference(name='Anita Hoffman')]...
DEBUG:spanner_graph_agent.spanner_graph_agent.tools.index_tools:Resolved reference_mappings: [ReferenceMapping(reference_in_user_query=UserProvidedPersonReference(name='Anita Hoffman'), canonical_references=[CanonicalPersonReference(id='p2005')])]
DEBUG:spanner_graph_agent.spanner_graph_agent.tools.langchain_tools:Input query: `What is the name of the company where Person(id='p2005') is employed?`




> Entering new SpannerGraphQAChain chain...
Executing gql:
GRAPH FinanceGraph
MATCH (person:Person {id: "p2005"}) -[:worksAt]-> (company:Company)
RETURN company.name AS company_name
Full Context:
[{'company_name': 'Grimes, Bowman and Greer'}, {'company_name': 'Anderson-Briggs'}, {'company_name': 'Davila-Robinson'}, {'company_name': 'Anderson, Aguirre and Eaton'}, {'company_name': 'Cisneros Inc'}, {'company_name': 'Brown-Villanueva'}, {'company_name': 'Clark-Jones'}, {'company_name': 'Vasquez-Randolph'}, {'company_name': 'Flynn LLC'}, {'company_name': 'Pena-Diaz'}, {'company_name': 'Jenkins LLC'}, {'company_name': 'Matthews-Vega'}, {'company_name': 'Horne-Russo'}, {'company_name': 'George Ltd'}, {'company_name': 'Rogers, Wilson and Bradshaw'}, {'company_name': 'Flores Inc'}, {'company_name': 'Lyons-Zavala'}, {'company_name': 'Smith and Sons'}, {'company_name': 'Guerra-Harper'}]

> Finished chain.
# 1-Hop Queries: Person to Company (Employment)
## Question
What is the name of the compa

DEBUG:spanner_graph_agent.spanner_graph_agent.tools.index_tools:Resolving: [UserProvidedPersonReference(name='Anita Hoffman')]...
DEBUG:spanner_graph_agent.spanner_graph_agent.tools.index_tools:Resolved reference_mappings: [ReferenceMapping(reference_in_user_query=UserProvidedPersonReference(name='Anita Hoffman'), canonical_references=[CanonicalPersonReference(id='p2005')])]
DEBUG:spanner_graph_agent.spanner_graph_agent.tools.langchain_tools:Input query: `Where does Person(id='p2005') work?`




> Entering new SpannerGraphQAChain chain...
Executing gql:
GRAPH FinanceGraph
MATCH (person:Person {id: "p2005"}) -[:worksAt]-> (company:Company)
RETURN company.name AS company_name
Full Context:
[{'company_name': 'Grimes, Bowman and Greer'}, {'company_name': 'Anderson-Briggs'}, {'company_name': 'Davila-Robinson'}, {'company_name': 'Anderson, Aguirre and Eaton'}, {'company_name': 'Cisneros Inc'}, {'company_name': 'Brown-Villanueva'}, {'company_name': 'Clark-Jones'}, {'company_name': 'Vasquez-Randolph'}, {'company_name': 'Flynn LLC'}, {'company_name': 'Pena-Diaz'}, {'company_name': 'Jenkins LLC'}, {'company_name': 'Matthews-Vega'}, {'company_name': 'Horne-Russo'}, {'company_name': 'George Ltd'}, {'company_name': 'Rogers, Wilson and Bradshaw'}, {'company_name': 'Flores Inc'}, {'company_name': 'Lyons-Zavala'}, {'company_name': 'Smith and Sons'}, {'company_name': 'Guerra-Harper'}]

> Finished chain.
# 1-Hop Queries: Person to Company (Employment)
## Question
Where does Anita Hoffman work

DEBUG:spanner_graph_agent.spanner_graph_agent.tools.index_tools:Resolving: [UserProvidedPersonReference(name='Anita Hoffman')]...
DEBUG:spanner_graph_agent.spanner_graph_agent.tools.index_tools:Resolved reference_mappings: [ReferenceMapping(reference_in_user_query=UserProvidedPersonReference(name='Anita Hoffman'), canonical_references=[CanonicalPersonReference(id='p2005')])]
DEBUG:spanner_graph_agent.spanner_graph_agent.tools.langchain_tools:Input query: `What is Person(id='p2005')'s job title?`




> Entering new SpannerGraphQAChain chain...
Executing gql:
GRAPH FinanceGraph
MATCH (person:Person {id: "p2005"}) -[works_at:worksAt]-> (:Company)
RETURN works_at.job_title AS job_title
Full Context:
[{'job_title': 'Banker'}, {'job_title': 'Retail merchandiser'}, {'job_title': 'Housing manager/officer'}, {'job_title': 'Architectural technologist'}, {'job_title': 'Designer, industrial/product'}, {'job_title': 'Teacher, secondary school'}, {'job_title': 'English as a second language teacher'}, {'job_title': 'Building control surveyor'}, {'job_title': 'Research officer, trade union'}, {'job_title': 'Primary school teacher'}, {'job_title': 'Chartered accountant'}, {'job_title': 'Energy engineer'}, {'job_title': 'Medical illustrator'}, {'job_title': 'Local government officer'}, {'job_title': 'Clinical biochemist'}, {'job_title': 'Medical physicist'}, {'job_title': 'Public affairs consultant'}, {'job_title': 'Lecturer, further education'}, {'job_title': 'Research scientist (maths)'}]

> Fi